In [1]:
import requests

In [3]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")
db = client.get_database("project3")

In [4]:
client_id ="OULV0HTDMPKCEWPASDNE5ZCAROMR3B0X2OKFEGTZC5AQOTIP" 
client_secret="X5QNCWMER3ABFZPTZATGX5HIFFURSAGFC0A4KOTR0XFADBZW"


In [5]:
url_query = 'https://api.foursquare.com/v2/venues/explore'

In [6]:
seville_cartuja = [37.402559152869856, -6.006329402641001]

In [7]:
def call_api (query, coordinates):
    parametros = {
        "client_id": client_id,
        "client_secret": client_secret,
        "v": "20220209",
        "ll": f"{coordinates[0]},{coordinates[1]}",
        "query": query,
        "limit": 200   
    }
    resp = requests.get(url_query, params=parametros).json()["response"]["groups"][0]["items"]
    return resp

In [9]:
mapa_nombre = ["venue", "name"]
mapa_latitud = ["venue", "location", "lat"]
mapa_longitud = ["venue", "location", "lng"]
mapa_direccion = ["venue", "location", "formattedAddress"]

In [10]:
def getFromDict(diccionario, mapa):
    from functools import reduce
    import operator
    return reduce(operator.getitem,mapa,diccionario)

In [11]:
def type_point(lista):
    return {"type": "Point", 
            "coordinates": lista
    }

In [12]:
def json_to_mongo (response, type_=""):
    json_to_mongo = []
    for dic in response:
        new_dic= {}
        new_dic["nombre"]= getFromDict(dic,mapa_nombre)
        lat = getFromDict(dic,mapa_latitud)
        lon = getFromDict(dic,mapa_longitud)
        new_dic["dirección"]= getFromDict(dic,mapa_direccion)
        new_dic["location"]=  type_point([lat,lon])
        new_dic["Type"] = type_
        json_to_mongo.append(new_dic)
    return json_to_mongo
        

In [13]:
import json
def export_dump (name, json_to_save) :
    with open(name, 'w') as f:
        json.dump(json_to_save,f)

In [14]:
json_vegan_ = call_api("vegan", seville_cartuja)
json_startup = call_api("Startup", seville_cartuja)
json_guarderias = call_api("guarderia", seville_cartuja)
json_colegios = call_api("colegio", seville_cartuja)
json_discotecas = call_api("discoteca", seville_cartuja)
json_starbucks = call_api("starbucks", seville_cartuja)

In [15]:
'''json_to_mongo(vegan_json)
 json_to_mongo(pubs_json)
json_to_mongo(guarderias_json)'''
json_mongo_IT = json_to_mongo(IT_json)

In [16]:
export_dump("startups.json",json_mongo_IT)

In [17]:
#db = client.get_database("restaurants")
restaurants = db.get_collection("restaurants")


In [18]:
#db.restaurants.create_index([("location", GEOSPHERE)])

In [24]:
#query = {"location": {"$near": {"$geometry": type_point(seville_cartuja),"$minDistance":100 ,"$maxDistance": 2000}}}

#list(restaurants.find(query))

In [20]:
query = [
   {
     '$geoNear': {
        'near':  [37.402559152869856, -6.006329402641001] ,
        'distanceField': "dist.calculated",
        'maxDistance': 2000,
        'distanceMultiplier': 6371,
        "spherical": True
     }
   }
]

In [21]:
geoloc = restaurants.aggregate(query)

In [22]:
milista= list(geoloc)

In [137]:
[x['dist'] for x in milista]

[{'calculated': 1.2694184342275248},
 {'calculated': 1.2700671489744895},
 {'calculated': 1.35735209930848},
 {'calculated': 1.4650009449775032},
 {'calculated': 1.5021327947317324},
 {'calculated': 1.5115961919192207},
 {'calculated': 1.7718311934017992},
 {'calculated': 1.8093710895103143},
 {'calculated': 2.1095812651266246},
 {'calculated': 2.1344208793755683},
 {'calculated': 2.146381912746296},
 {'calculated': 2.214905251319349},
 {'calculated': 2.264377942100668},
 {'calculated': 2.6187797059889144},
 {'calculated': 2.7063000055375865},
 {'calculated': 2.8929602669769054},
 {'calculated': 4.5644338752369515}]